In [1]:
from pyquil import Program, get_qc
from pyquil.gates import *
import pyquil.api as api
from pyquil.paulis import *
import numpy as np
import math

In [2]:
qvm = api.QVMConnection()
np.random.seed(seed=0)

In [3]:
# hopping term: a_p^dagger a_q
t = 1
hamt = (-t / 2) * (sX(0)*sX(1) + sX(2)*sX(3) + sY(0)*sY(1) + sY(2)*sY(3))

# interaction term: a_p^dagger a_q^dagger a_q a_p
U1 = 1
U2 = 1
hamU = (U1 / 4) * (sI(3) - sZ(0) - sZ(2) + sZ(0)*sZ(2))
hamU += (U2 / 4) * (sI(0) - sZ(1) - sZ(3) + sZ(1)*sZ(3))

# number term: a_p^dagger a_p
e1 = 2
e2 = 0
hame = (e1 / 2) * (2*sI() - sZ(0) - sZ(2))
hame += (e2 / 2) * (2*sI() - sZ(1) - sZ(3))

hamiltonian = hamt + hamU + hame
print(hamt)
print(hamU)
print(hame)

(-0.5+0j)*X0*X1 + (-0.5+0j)*X2*X3 + (-0.5+0j)*Y0*Y1 + (-0.5+0j)*Y2*Y3
(0.5+0j)*I + (-0.25+0j)*Z0 + (-0.25+0j)*Z2 + (0.25+0j)*Z0*Z2 + (-0.25+0j)*Z1 + (-0.25+0j)*Z3 + (0.25+0j)*Z1*Z3
(2+0j)*I + (-1+0j)*Z0 + (-1+0j)*Z2


In [4]:
from grove.pyvqe.vqe import VQE
from scipy.optimize import minimize
import numpy as np

vqe_inst = VQE(minimizer=minimize, minimizer_kwargs={'method': 'nelder-mead'})

In [5]:
from qucochemistry.utils import pyquilpauli_to_qubitop
from openfermion.transforms import get_sparse_operator
h = get_sparse_operator(pyquilpauli_to_qubitop(hamiltonian))
np.savetxt('h.txt', np.real(h.todense()), '%-2d', ', ')
if h.shape[0] > 1024:
    [w, _] = sp.sparse.linalg.eigsh(h, k=1)
else:
    [w, _] = np.linalg.eigh(h.todense())
w

array([-0.41421356, -0.41421356, -0.14133612,  0.        ,  2.        ,
        2.        ,  2.        ,  2.41421356,  2.41421356,  2.51513805,
        2.58578644,  2.58578644,  5.41421356,  5.41421356,  5.62619807,
        6.        ])

In [6]:
def var_ansatz(params):
    S = 2
    p = Program()
    
    # put initial state in the sector
    
    # Product state -- 1/sqrt(2) (|1001> + |0110>)
    p.inst(H(0))
    p.inst(CNOT(0, 1))
    p.inst(CNOT(1, 2))
    p.inst(X(1))
    p.inst(CNOT(2, 3))
    p.inst(X(2))
    
    # |1001>
    #p.inst(X(0))
    #p.inst(X(3))
    
    # HF state -- 1/2 (|0101> + |0110> + |1001> + |1010>)
    """
    p.inst(H(0))
    p.inst(H(2))
    p.inst(CNOT(0, 1))
    p.inst(CNOT(2, 3))
    p.inst(X(0))
    p.inst(X(2))
    """   
    for i in range(S):
        # building U_u
        p.inst(RZ(U1*params[3*i]/4, 0))
        p.inst(RZ(U2*params[3*i]/4, 1))
        p.inst(RZ(U1*params[3*i]/4, 2))
        p.inst(RZ(U2*params[3*i]/4, 3))
        p.inst(CNOT(0, 2))
        p.inst(RZ(-U1*params[3*i]/4, 2))
        p.inst(CNOT(0, 2))
        p.inst(CNOT(1, 3))
        p.inst(RZ(-U2*params[3*i]/4, 3))
        p.inst(CNOT(1, 3))
        
        #building U_\epsilon
        p.inst(RZ(e2*params[3*i+2], 1))
        p.inst(RZ(e2*params[3*i+2], 3))
        p.inst(RZ(e1*params[3*i+2], 0))
        p.inst(RZ(e1*params[3*i+2], 2))
        
        # building U_t
        p.inst(H(0))
        p.inst(H(1))
        p.inst(H(2))
        p.inst(H(3))
        p.inst(CNOT(0, 1))
        p.inst(CNOT(2, 3))
        p.inst(RZ(params[3*i+1], 1))
        p.inst(RZ(params[3*i+1], 3))
        p.inst(CNOT(0, 1))
        p.inst(CNOT(2, 3))
        p.inst(H(0))
        p.inst(H(1))
        p.inst(H(2))
        p.inst(H(3))
        p.inst(RX(-np.pi/2, 0))
        p.inst(RX(-np.pi/2, 1))
        p.inst(RX(-np.pi/2, 2))
        p.inst(RX(-np.pi/2, 3))
        p.inst(CNOT(0, 1))
        p.inst(CNOT(2, 3))
        p.inst(RZ(params[3*i+1], 1))
        p.inst(RZ(params[3*i+1], 3))
        p.inst(CNOT(0, 1))
        p.inst(CNOT(2, 3))
        p.inst(RX(-np.pi/2, 0))
        p.inst(RX(-np.pi/2, 1))
        p.inst(RX(-np.pi/2, 2))
        p.inst(RX(-np.pi/2, 3))
        
        # building U_u
        p.inst(RZ(U1*params[3*i]/4, 0))
        p.inst(RZ(U2*params[3*i]/4, 1))
        p.inst(RZ(U1*params[3*i]/4, 2))
        p.inst(RZ(U2*params[3*i]/4, 3))
        p.inst(CNOT(0, 2))
        p.inst(RZ(-U1*params[3*i]/4, 2))
        p.inst(CNOT(0, 2))
        p.inst(CNOT(1, 3))
        p.inst(RZ(-U2*params[3*i]/4, 3))
        p.inst(CNOT(1, 3))
        
    return p

In [7]:
from scipy.optimize import minimize
import random

def expectation(x):
    """
    Expectation value for parameters, x
    """
    return vqe_inst.expectation(var_ansatz(x), hamiltonian, None, qvm)

def scipy_minimize(x):
    """
    Minimizes expectation value using parameters found in greedy search as starting point
    Powell's conjugate direction method
    """
    return minimize(expectation, x, method='powell')

def greedy_noisy_search(initial_state):
    """
    Slightly perturb the values of the points, accepting whenever this results in a lower energy
    Total of 150 evaulations of the energy
    Change step size after 30 evaluations based on number of acceptances in previous trial group
    """
    params = initial_state
    min_energy = vqe_inst.expectation(var_ansatz(initial_state), hamiltonian, None, qvm)

    def random_point(dim, step):
        """
        Generates a random point on the perimeter of a circle with radius, step
        """
        coords = [random.gauss(0, 1) for i in range(dim)]
        norm = math.sqrt(sum([i**2 for i in coords]))
        coords = [(i / norm) * step for i in coords]
        return coords

    step = 0.1
    acceptances = 0

    # Five groups of 30 trials
    for i in range(5):
        acceptances = 0
        for j in range(30):
            
            # Slightly perturb the values of the points
            coords = random_point(len(initial_state), step)
            temp_params = [sum(x) for x in zip(params, coords)]
            
            # Calculate expectation value with new parameters
            temp_energy = vqe_inst.expectation(var_ansatz(temp_params), hamiltonian, None, qvm)
            
            # Greedily accept parameters that result in lower energy
            if (temp_energy < min_energy):
                min_energy = temp_energy
                params = temp_params
                acceptances += 1
        # Update step size
        step *= (acceptances / 15)

    return {'x': params, 'energy': min_energy, 'step': step}

def global_variational(S):
    """
    Complete a round of greedy and Powell's optimization for six randomly chosen points
    Further optimizes the best point
    """    
    points = [np.random.rand(S*3)*0.2 for i in range(6)]
    greedy_results = [greedy_noisy_search(x) for x in points]
    print("Done with greedy search")
    results = [scipy_minimize(i['x']) for i in greedy_results]
    print("Done with Powell's")
    res = min(results, key=lambda x:x.fun)
    print(res)
    # For the chosen point, we continue optimizing until we cannot find improvement
    res_copy = res
    energy = res_copy.fun
    while (True):
        result = greedy_noisy_search(res_copy.x)
        temp_res = scipy_minimize(result['x'])
        temp_energy = temp_res.fun
        print(temp_energy)

        # Optimizing will usually find an energy ~1e-6 lower
        # Stop it eventually
        tolerance = 0.00001
        if (abs(temp_energy - energy) > tolerance):
            res_copy = temp_res
            energy = temp_energy
        else:
            break
    return res_copy

In [8]:
# make sure to change this S to match the one in the ansatz
S = 2
res = global_variational(S) ccc
print(res)

Done with greedy search
Done with Powell's
   direc: array([[ 0.16897664,  0.03034708,  8.9660054 , -0.05959522, -0.17853741,
         0.07534543],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.80983563, -0.02011907,  7.9413278 , -0.46039383, -0.15321776,
         0.06757232]])
     fun: array(-0.14133609)
 message: 'Optimization terminated successfully.'
    nfev: 486
     nit: 7
  status: 0
 success: True
       x: array([ 3.20072299,  0.41300431, 31.07744963, -0.34376761, -0.70619824,
        0.30556663])
-0.14133611564312326
   direc: array([[ 0.16897664,  0.03034708,  8.9660054 , -0.05959522, -0.17853741,
         0.07534543],
       [ 0.    

In [9]:
def op(s1, s2):
    # returns a PauliSum representing a_s1^daggar a_s2
    def sigma_plus(s):
        # returns sigma^+ operator on side s
        # sigma^+ = 1/2(X + iY)
        return PauliTerm('X', s, 0.5) + PauliTerm('Y', s, complex(0.0+0.5j))
    def sigma_minus(s):
        # returns sigma^- operator on side s
        # sigma^- = 1/2(X - iY)
        return PauliTerm('X', s, 0.5) - PauliTerm('Y', s, complex(0.0+0.5j))
    
    # for all sites in between site one (s1) and site two (s2), multiply by sigma z
    z = sI()
    for i in range(s1+1, s2):
        z *= PauliTerm('Z', i)
        
    return sigma_minus(s1) * z * sigma_plus(s2)

In [10]:
 # building the single particle density matrix (spdm) 
spdm = np.zeros((N*2,N*2))
for i in range(N*2):
    for k in range(N*2):
        # building an operator and vqe for each i, j        
        # element (j, i) in the spdm is a_i^dagger a_j
        spdm[k, i] = vqe_inst.expectation(var_ansatz(res.x), op(i, k), None, qvm)
print(np.around(spdm, decimals=2))